In [3]:
import numpy as np 
import matplotlib.pyplot as plt

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
palette = sns.color_palette('bright')
palette

[(0.00784313725490196, 0.24313725490196078, 1.0),
 (1.0, 0.48627450980392156, 0.0),
 (0.10196078431372549, 0.788235294117647, 0.2196078431372549),
 (0.9098039215686274, 0.0, 0.043137254901960784),
 (0.5450980392156862, 0.16862745098039217, 0.8862745098039215),
 (0.6235294117647059, 0.2823529411764706, 0.0),
 (0.9450980392156862, 0.2980392156862745, 0.7568627450980392),
 (0.6392156862745098, 0.6392156862745098, 0.6392156862745098),
 (1.0, 0.7686274509803922, 0.0),
 (0.0, 0.8431372549019608, 1.0)]

In [ ]:
#RK4 METHOD 

In [48]:
class Planet:
    def __init__ (self,PlanetName):
        self.PlanetName = PlanetName
        self.position = self.__position() #Nowstate position
        self.velocity = self.__velocity() #Nowstate velocity
        self.mu = self.__mu()
        self.distance = self.__distance() #Distance from the Sun 
        _AU = 1.496e+11                    #1AU to m 
        _hour = 3600                       #hour to second
        _ss = 7.71604938e-8                #sec^2 to hour^2
    
class PlanetMercury(Planet):
    def __distance(self):
        return float(1.496e+11*0.39)       #aphelion position
    def __position(self):
        return float(1.496e+11*0.39)       #initial position
    def __velocity(self):
        return 47.4e3
    def __mu(self):                        #Mu = G*M [m^3/s^2]
        return 2.2032e13

class PlanetVenus(Planet):
    def __distance(self):
        return float(1.496e+11*0.723)
    def __position(self):
        return float(1.496e+11*0.723)
    def __velocity(self):
        return 35.1e3
    def __mu(self):
        return 3.24859e14

class PlanetEarth(Planet):
    def __distance(self):
        return float(1.496e+11*1)
    def __position(self):
        return float(1.496e+11*1)
    def __velocity(self):
        return 29.8e3
    def __mu(self):
        return 3.986004418e14

class PlanetMars(Planet):
    def __distance(self):
        return float(1.496e+11*1.524)
    def __position(self):
        return float(1.496e+11*1.524)
    def __velocity(self):
        return 24.1e3
    def __mu(self):
        return 4.282837e13

class PlanetJupiter(Planet):
    def __distance(self):
        return float(1.496e+11*5.203)
    def __position(self):
        return float(1.496e+11*5.203)
    def __velocity(self):
        return 13.1e3
    def __mu(self):
        return 1.26686534e17

class PlanetSaturn(Planet):
    def __distance(self):
        return float(1.496e+11*9.539)
    def __position(self):
        return float(1.496e+11*9.539)
    def __velocity(self):
        return 9.7e3
    def __mu(self):
        return 3.7931187e16

class PlanetUranus(Planet):
    def __distance(self):
        return float(1.496e+11*19.18)
    def __position(self):
        return float(1.496e+11*19.18)
    def __velocity(self):
        return 6.8e3
    def __mu(self):
        return 5.793939e15

class PlanetNeptune(Planet):
    def __distance(self):
        return float(1.496e+11*30.06)
    def __position(self):
        return float(1.496e+11*30.06)
    def __velocity(self):
        return 5.4e3
    def __mu(self):
        return 6.836529e15

class PlanetPluto(Planet):
    def __distance(self):
        return float(1.496e+11*39.53)
    def __position(self):
        return float(1.496e+11*39.53)
    def __velocity(self):
        return 4.7e3
    def __mu(self):
        return 8.71e11

class CometHalley(Planet):
    def __distance(self):
        return float(1.496e+11*35.3)
    def __position(self):
        return float(1.496e+11*35.3)
    def __velocity(self):
        return 1000
    def __mu(self):
        return 6.670e3*2.2

In [36]:
def planets(PlanetName):
    if PlanetName == 1:
        return PlanetMercury('Mercury')

In [40]:
M = PlanetMercury('Mercury')

In [47]:
M.velocity

47400.0